In [2]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install nltk
!pip install torch
!pip install bert_score
!pip install torchtext
!pip install language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.

In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf
import torch

In [4]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [5]:
# Define dataset path
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [6]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [7]:
from transformers import pipeline
pipe = pipeline("text2text-generation", model="facebook/m2m100_418M")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M").to(device)

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [8]:
translatedEngHin = []
for index, (eng, hin) in enumerate(zip(english_sentences, hindi_sentences)):
    encoded_en = tokenizer(eng, return_tensors="pt", padding=True, truncation=True).to(device)
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("hi")).to(device)
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    print(f"Index: {index+1}\nEnglish: {eng}\nHindi (Original): {hin}\nHindi (Translated): {translation}\n")
    print("------------------------------")
    translatedEngHin.append(translation)

comparison = pd.DataFrame({'Original': english_sentences, 'Translated': translatedEngHin})
comparison.to_csv('/content/IITB_Translated_EngHin.csv', index=False)

Streaming output truncated to the last 5000 lines.

------------------------------
Index: 168
English: _ Edit Bookmarks...
Hindi (Original): पुस्तक चिह्न संपादित करें (_ E)... 
Hindi (Translated): संपादित करें Bookmarks...

------------------------------
Index: 169
English: Manage bookmarks.
Hindi (Original): पुस्तक-चिह्नों को प्रबंधित करें। 
Hindi (Translated): किताबों का प्रबंधन।

------------------------------
Index: 170
English: Edit Bookmarks...
Hindi (Original): पुस्तक चिह्नों को संपादित करें... 
Hindi (Translated): संपादित करें Bookmarks...

------------------------------
Index: 171
English: Title
Hindi (Original): शीर्षक
Hindi (Translated): शीर्षक

------------------------------
Index: 172
English: Application
Hindi (Original): अनुप्रयोग
Hindi (Translated): आवेदन

------------------------------
Index: 173
English: Path
Hindi (Original): पथ
Hindi (Translated): रास्ते

------------------------------
Index: 174
English: Add Bookmark...
Hindi (Original): पुस्तक-चिह्न जोड़ें... 
Hin

In [9]:
# BLEU Score
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[hindi] for hindi in hindi_sentences]  # Create reference list of lists
candidates = translatedEngHin  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.3636


In [10]:
# STS Score

!pip install sentence-transformers

# STS Score using SentenceTransformer
from sentence_transformers import SentenceTransformer, util
import torch

# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(hindi_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedEngHin, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.9 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.7286


In [11]:
# CHRF Score

!pip install sacrebleu
from sacrebleu.metrics import CHRF

# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedEngHin, [hindi_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")


CHRF score: 28.9848


In [12]:
# METEOR score

import nltk
from nltk.translate.meteor_score import meteor_score

# Download required NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
hindi_sentences_tokenized = [nltk.word_tokenize(sent) for sent in hindi_sentences]
translatedEngHin_tokenized = [nltk.word_tokenize(sent) for sent in translatedEngHin]

# Calculate METEOR score
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(hindi_sentences_tokenized, translatedEngHin_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR score: 0.2059


In [13]:
import nltk
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Download necessary NLTK data
nltk.download('punkt')

def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
hindi_model = train_model(hindi_sentences)
english_model = train_model(translatedEngHin)

# Calculate perplexities
hindi_perplexity = calculate_perplexity(hindi_model, hindi_sentences)
english_perplexity = calculate_perplexity(english_model, translatedEngHin)

print(f"Hindi Perplexity: {hindi_perplexity:.4f}")
print(f"Translated English Perplexity: {english_perplexity:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Hindi Perplexity: 11.6234
Translated English Perplexity: 11.6886
